# Lab 4 Instructions and Exercises (Big Data in Statistics)

## Introduction

These notes contain instructions and questions for the labs portion of the Big Data in Statistics module. Within this document, command-line steps are presented as follows:

In [ ]:
hadoop fs -put data /user/mark/repository/data

All commands will be in a separate grey "cell" (as above).
<br><br>
Exercises will be listed as a bulleted item and italicized. For example:

<ul><li><i>Create a new directory in your HDFS home directory called sample. Upload data.csv into the sample directory on HDFS.</i></li></ul>

To follow real-world development practices, you will be using configuration control software git, and internet based repositories on <a href="http://github.com">github.com</a>. Instructions will be provided on how to use these tools during the exercises.

## Objectives

In this lab you will be expected to achieve the following:
<ol>
<li>Perform Map Reduce operations in Spark
<li>Understand and use mllib for statistical analysis of data
</ol>

## Exercises

### Exercise 1

Connect to the Spark REPL.
<ul>
<li><i>Following week 3 exercises, create two RDDs (heathrowData and wickairportData) of type org.apache.spark.rdd.RDD[TemperatureRecord]. NB: Remember to remove header lines and missing data lines.
<li>Using the function that you have created to remove missing data (or otherwise), make a note of the year(s) and month(s) of the missing data records.</i>
</ul>

### Exercise 2

It is possible to replicate Map Reduce processing in Spark. Consider the following statement:

In [ ]:
scala> val heathrowAverageRain = heathrowData.map(x => (x.year,x.rain)).aggregateByKey((0.0, 0.0))((acc, value) => (acc._1 + value, acc._2 + 1), (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)).mapValues(sumCount => 1.0 * sumCount._1 / sumCount._2).sortBy(_._1)

The RDD heathrowAverageRain is of type org.apache.spark.rdd.RDD[(Long, Double)], with the first element representing the year and the second element representing the average rainfall in that year. The RDD is sorted by year, as shown by the final function call.

There are three key components used in this transformation.
<ol>
<li>The map function extracts the (year, rainfall) <key,value> pairs. The output type of this transformation is org.apache.spark.rdd.RDD[(Long, Float)], with an entry per line in the data.
<li>The aggregateByKey function computes a pair of two values for each key (year); the first is the sum of rainfall, and the second is a count of the number of elements. These counts are both initialized with 0.0. The output type of the second transformation is org.apache.spark.rdd.RDD[(Long, (Double, Double))], linking the key (year) to the two aforementioned aggregated values.
<li>The third function, mapValues, computes the average rainfall for each key by combining the two Double values.
</ol>

<ul>
<li><i>Using a similar transformation to that above, compute the average monthly max temperature for both airports (heathrowAverageTMax and wickAverageTMax).</i>
</ul>

### Exercise 3

<ul>
<li><i>Using the appropriate information contained on the following webpage: http://spark.apache.org/docs/latest/programming-guide.html#transformations, join the heathrowData and wickairportData datasets (using the join operation) to create an RDD called combinedData. The output should be of type org.apache.spark.rdd.RDD[((Int, Long), (Float, Float))], where the tuple corresponds to the (year, month) and the second tuple corresponds to the (Heathrow.TMax, Wick.TMax).</i>
</ul>

### Exercise 4

It is possible to use mllib to compute basic summary statistics of the data using the following exemplar commands:

In [ ]:
scala> import org.apache.spark.mllib.linalg.Vector

In [ ]:
scala> import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}

In [ ]:
scala> val summary: MultivariateStatisticalSummary = Statistics.colStats(observations)

In [ ]:
scala> println(summary.mean)

In [ ]:
scala> println(summary.variance)

Note that <i>observations</i> is an RDD[Vector], which can be constructed by converting the input array into a dense Vector (see http://spark.apache.org/docs/latest/mllib-data-types.html)

<ul>
<li><i>Compute summary statistics for each monthly max temperature data (summary statistics for each airport), using the appropriate columns of the joined RDD (combinedData) from the previous exercise.</i>
</ul>

### Exercise 5

The following command produces the Pearson correlation coefficient for two data series (labelled seriesX and seriesY here)

In [ ]:
scala> import org.apache.spark.mllib.stat.Statistics

In [ ]:
scala> val correlation = Statistics.corr(seriesX, seriesY, "pearson")

<ul>
<li><i>Compute the Pearson correlation coefficient for the two average max temperature datasets computed in Exercise 2 (heathrowAverageTMax and wickAverageTMax). What does this tell you about the data?</i>
</ul>

### Exercise 6

The following commands demonstrate how to estimate the parameters of a linear regression model:

In [ ]:
scala> import org.apache.spark.mllib.regression.LabeledPoint

In [ ]:
scala> import org.apache.spark.mllib.regression.LinearRegressionModel

In [ ]:
scala> import org.apache.spark.mllib.regression.LinearRegressionWithSGD

In [ ]:
scala> import org.apache.spark.mllib.linalg.Vectors

In [ ]:
scala> val data = sc.textFile("data.txt")

In [ ]:
scala> val parsedData = data.map { line =>
  val parts = line.split(',')
  LabeledPoint(parts(0).toDouble, Vectors.dense(parts(1).split(' ').map(_.toDouble)))
}.cache()

Now to build the model:

In [ ]:
scala> val numIterations = 100

In [ ]:
scala> val model = LinearRegressionWithSGD.train(parsedData, numIterations)

Evaluate the model on training examples and compute training error:

In [ ]:
scala> val valuesAndPreds = parsedData.map { point =>
  val prediction = model.predict(point.features)
  (point.label, prediction)
}

In [ ]:
scala> val MSE = valuesAndPreds.map{case(v, p) => math.pow((v - p), 2)}.mean()

In [ ]:
scala> println("training Mean Squared Error = " + MSE)

<ul>
<li><i>Estimate the parameters of a linear regression model using the combined data, with max temperature for Heathrow airport as the input and max temperature for Wick as the output variable.</i>
</ul>

HINT: The default step size is too large for this particular example. It is possible to reduce the step size to a smaller value (0.01 is recommended). The train function, and its input parameters, are described here: https://spark.apache.org/docs/1.5.2/api/java/org/apache/spark/mllib/regression/LinearRegressionWithSGD.html#train(org.apache.spark.rdd.RDD, int, double)

<ul>
<li><i>Using the trained model, predict the values of the max temperature data for Wick airport using the max temperature value for the Heathrow airport at the year and month of the missing Wick data points (using the output from Exercise 1b).</i>
</ul>